In [1]:
import azureml.core
import sklearn
from azureml.core import Experiment, ScriptRunConfig, Environment, Workspace
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline 
from azureml.pipeline.core import PipelineRun

In [2]:
ws = Workspace.from_config()
#exp = Experiment(ws,'predictive_maintenance_experiment')

In [3]:
env = Environment('pm_env')
env.python.user_managed_dependencies = False #lar azure håndtere dependencies hvis noe mangler
env.docker.enabled = True
#azureml.core.runconfig.DockerConfiguration['use_docker'] = True
packages = ['scikit-learn==0.23.2','azureml-defaults','joblib','numpy','pandas','tensorflow',
'xgboost','pyspark','azureml-dataprep[pandas]']

for package in packages:
    env.python.conda_dependencies.add_pip_package(package)

my_env = env.register(workspace=ws)

'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
#run configuration
from azureml.core import RunConfiguration
train_compute_target = ws.compute_targets["MLhubComp"]
pipe_run_config = RunConfiguration()
pipe_run_config.environment = my_env
pipe_run_config.target = train_compute_target

In [26]:
#create pipeline
train = ws.datasets.get('train_FD001')
test = ws.datasets.get('test_FD001')
rul = ws.datasets.get('RUL_FD001.txt')

transformed_train_folder = PipelineData(name = "transform_train_folder",datastore=ws.get_default_datastore())
transformed_test_folder = PipelineData(name = "transform_test_folder",datastore=ws.get_default_datastore())

train_step = PythonScriptStep(name = "transform data",source_directory='',script_name='pm_data_cleaning.py',
arguments=['--train-data',train.as_named_input('raw_train_data'),
'--test-data',test.as_named_input('raw_test_data'),
'--rul-data',rul.as_named_input('rul_data'), #lager navnet raw_data til å peke til house_df dataen
'--cleanedtrain-data',transformed_train_folder,
'--cleanedtest-data',transformed_test_folder], #vilkårlig at det står input og prepped data. 
#kan stå hva som helst, er bare viktig at det er "--" foran og at samme ord blir brukt i scriptet house data cleaning
outputs=[transformed_train_folder,transformed_test_folder],
compute_target=train_compute_target,
runconfig=pipe_run_config,
allow_reuse=True)

#run the training script
model_step = PythonScriptStep(name = 'train and register models',
source_directory='',script_name='pm_train_model.py',
arguments= ['--training-folder',transformed_train_folder,
'--test-folder',transformed_test_folder], 
inputs = [transformed_train_folder,transformed_test_folder], #her defineres at inputte til dette scriptet er outputte til det forrige 
compute_target= train_compute_target,
runconfig= pipe_run_config,
allow_reuse= True)

In [27]:
pipeline_steps = [train_step,model_step]
pipeline = Pipeline(workspace=ws,steps=pipeline_steps)

exp = Experiment(ws,'pm-finalpipe')
pipe_run = exp.submit(pipeline,regenerate_outputs = True)

Created step transform data [cc333467][8d064b12-b438-4ec0-b75f-f7f2f2bb78e0], (This step will run and generate new outputs)
Created step train and register models [29a5408a][d01004cc-e23d-47e7-9320-77469d00aede], (This step will run and generate new outputs)
Submitted PipelineRun b887e868-dee2-448d-9bc0-5b6cd196c852
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/b887e868-dee2-448d-9bc0-5b6cd196c852?wsid=/subscriptions/bb345bae-e66f-461d-a1fe-67219d54a0f3/resourcegroups/rg-lara-veylit/workspaces/mlhub&tid=40cc2915-e283-4a27-9471-6bdd7ca4c6e1


In [19]:
#RunDetails(pipe_run).show()
pipe_run.wait_for_completion(show_output=True)

PipelineRunId: 1136f40a-d8a0-4fb9-8b33-b83375e72743
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/1136f40a-d8a0-4fb9-8b33-b83375e72743?wsid=/subscriptions/bb345bae-e66f-461d-a1fe-67219d54a0f3/resourcegroups/rg-lara-veylit/workspaces/mlhub&tid=40cc2915-e283-4a27-9471-6bdd7ca4c6e1

PipelineRun Execution Summary
PipelineRun Status: Failed
{'runId': '1136f40a-d8a0-4fb9-8b33-b83375e72743', 'status': 'Failed', 'startTimeUtc': '2022-05-09T13:34:50.435078Z', 'endTimeUtc': '2022-05-09T13:44:02.251115Z', 'services': {}, 'properties': {'azureml.runsource': 'azureml.PipelineRun', 'runSource': 'SDK', 'runType': 'SDK', 'azureml.parameters': '{}', 'azureml.continue_on_step_failure': 'False', 'azureml.pipelineComponent': 'pipelinerun'}, 'inputDatasets': [], 'outputDatasets': [], 'logFiles': {'logs/azureml/executionlogs.txt': 'https://mlhub1649667653.blob.core.windows.net/azureml/ExperimentRun/dcid.1136f40a-d8a0-4fb9-8b33-b83375e72743/logs/azureml/executionlogs.txt?sv=2019-07-07&sr=b

'Failed'

# publish pipeline

In [17]:
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))
experiment_name = 'pm-finalpipe'
pipeline_experiment = ws.experiments.get(experiment_name)


pipe_run = list(pipeline_experiment.get_runs())[0]
pipeline_run = PipelineRun(pipeline_experiment, pipe_run.id)

Ready to use Azure ML 1.40.0 to work with mlhub
